In [ ]:
!pip install python-terrier
#!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

     |████████████████████████████████| 97 kB 3.3 MB/s 
     |████████████████████████████████| 1.1 MB 32.5 MB/s 
     |████████████████████████████████| 69 kB 7.6 MB/s 
     |████████████████████████████████| 255 kB 52.6 MB/s 
     |████████████████████████████████| 41 kB 274 kB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 126 kB 55.8 MB/s 
     |████████████████████████████████| 6.4 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 294 kB 42.6 MB/s 
     |████████████████████████████████| 291 kB 59.9 MB/s 
     |████████████████████████████████| 59 kB 5.2 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=806c665a8e26438c6224ef53d2984da9dcbe68f9839ca3abacfb4e7ce4764299
  Stored in directory: /root/.cac

In [ ]:
import torch
import time as time
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [ ]:
use_cuda

False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pt.get_dataset("trec-deep-learning-passages")
print(dataset.get_topics("test-2019"))

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

         qid                                              query
0    1108939                  what slows down the flow of blood
1    1112389             what is the county for grand rapids mn
2     792752                                     what is ruclip
3    1119729  what do you do when you have a nosebleed from ...
4    1105095                  where is sugar lake lodge located
..       ...                                                ...
195   146187  difference between a mcdouble and a double che...
196   634428                            what does chs stand for
197  1121986     what are the effects of having low blood sugar
198   321441                 how much is a us postal stamp cost
199   532603                   university of dubuque enrollment

[200 rows x 2 columns]


In [ ]:
# Iterator for msmarco passage
def msmarco_generate():

    dataset = pt.get_dataset("trec-deep-learning-passages")
    print("Hi")
    with pt.io.autoopen('./drive/My Drive/Colab Notebooks/collection.tsv', 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}


msmarco_generate()


<generator object msmarco_generate at 0x7f0a54f53f50>

In [ ]:
#Opening indexed files

msmarco_generate()


indexref = pt.IndexRef.of("./drive/My Drive/Colab Notebooks/indexes/msmarco-passage")
index = pt.IndexFactory.of("./drive/My Drive/Colab Notebooks/indexes/msmarco-passage")

#msmarco_generate()
dataset


RemoteDataset for msmarco_passage, with ['corpus', 'index', 'topics', 'tars', 'qrels', 'info_url', 'corpus_iter']

In [ ]:
# Batch Retrievalfor BM25 
BM25_baseline = pt.BatchRetrieve(index, wmodel="BM25", verbose=True)

In [ ]:
# Evaluation

start_time = time.time()

result = pt.Experiment([BM25_baseline], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map"])

print(f'Time taken : {time.time() - start_time}')

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

BR(BM25):   0%|          | 0/200 [00:00<?, ?q/s]

Time taken : 39.02860498428345


In [ ]:
# Topic wise analysis

start_time = time.time()

result_per_query = pt.Experiment([BM25_baseline], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map"],
                                 perquery=True)

print(f'Time taken : {time.time() - start_time}')

BR(BM25):   0%|          | 0/200 [00:00<?, ?q/s]

Time taken : 17.950525760650635


/usr/local/lib/python3.7/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [ ]:
result_per_query = result_per_query[result_per_query["value"].notnull()]
print(result_per_query)

        name      qid      measure     value
27  BR(BM25)  1037798          map  0.109533
28  BR(BM25)  1037798   recip_rank  0.333333
29  BR(BM25)  1037798  ndcg_cut_10  0.152866
63  BR(BM25)   104861          map  0.356783
64  BR(BM25)   104861   recip_rank  1.000000
..       ...      ...          ...       ...
31  BR(BM25)   915593   recip_rank  1.000000
32  BR(BM25)   915593  ndcg_cut_10  0.417021
39  BR(BM25)   962179          map  0.058701
40  BR(BM25)   962179   recip_rank  0.025000
41  BR(BM25)   962179  ndcg_cut_10  0.000000

[129 rows x 4 columns]


# L2R part below


In [ ]:
topics = dataset.get_topics("test-2019")
qrels = dataset.get_qrels("test-2019")
print("printing qrels")
print(qrels)

print("printing topics")
print(topics)
#train_topics, valid_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

printing qrels
          qid    docno  label
0       19335  1017759      0
1       19335  1082489      0
2       19335   109063      0
3       19335  1160863      0
4       19335  1160871      0
...       ...      ...    ...
9255  1133167  8839920      2
9256  1133167  8839922      2
9257  1133167   944810      0
9258  1133167   949411      0
9259  1133167   977421      0

[9260 rows x 3 columns]
printing topics
         qid                                              query
0    1108939                  what slows down the flow of blood
1    1112389             what is the county for grand rapids mn
2     792752                                     what is ruclip
3    1119729  what do you do when you have a nosebleed from ...
4    1105095                  where is sugar lake lodge located
..       ...                                                ...
195   146187  difference between a mcdouble and a double che...
196   634428                            what does chs stand for
197  112

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# print(pt.pipelines)

# BaselineLTR = fbr >> pt.pipelines.LTR_pipeline(RandomForestRegressor(n_estimators=400))
# # BaselineLTR.fit(train_topics, qrels)

# # results = pt.pipelines.Experiment([PL2, BaselineLTR], test_topics, qrels, ["map"], names=["PL2 Baseline", "LTR Baseline"])
# # results

In [ ]:
from sklearn.ensemble import RandomForestRegressor

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = bm25 >> (tf ** pl2)

pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2"])

rf = RandomForestRegressor(n_estimators=400)
rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(topics, qrels)
pt.Experiment([bm25, rf_pipe], topics, qrels, ["map"], names=["BM25 Baseline", "LTR"])

,name,map
0,BM25 Baseline,0.370014
1,LTR,0.492018


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeb7fcb4-84cb-4f4a-9830-9d1fcbc47e1d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>